# Test Claude 3.5 Sonnet Vision API for Flyer Extraction

This notebook validates the "Tier 3" approach using Claude's vision capabilities.

**Goals:**
- Test extraction accuracy
- Measure cost per flyer
- Evaluate quality and confidence
- Compare against manual extraction

## Setup

In [ ]:
import anthropic
import base64
import json
import time
from pathlib import Path
from IPython.display import Image, display
import os

In [ ]:
# Set your API key (or use environment variable ANTHROPIC_API_KEY)
# Get your key from: https://console.anthropic.com/

API_KEY = os.getenv('ANTHROPIC_API_KEY')  # or set directly: "sk-ant-..."

if not API_KEY:
    print("⚠️  Warning: ANTHROPIC_API_KEY not set!")
    print("   Set it with: export ANTHROPIC_API_KEY='your-key'")
    print("   Or set API_KEY variable in this cell")
else:
    print("✅ API key loaded")

## 1. Prepare Test Image

Options:
1. Screenshot a flyer from Flipp and save to `data/raw/samples/`
2. Use `download_sample_flyer.py` to get a sample
3. Convert existing PDF to images using `convert_by_pymupdf.ipynb`

In [ ]:
# Set path to your test image
IMAGE_PATH = Path("../data/raw/samples/test_flyer.png")

# Create directory if it doesn't exist
IMAGE_PATH.parent.mkdir(parents=True, exist_ok=True)

if IMAGE_PATH.exists():
    print(f"✅ Image found: {IMAGE_PATH}")
    print(f"📏 Size: {IMAGE_PATH.stat().st_size / 1024:.1f} KB")
    
    # Display the image
    display(Image(filename=IMAGE_PATH, width=600))
else:
    print(f"❌ Image not found: {IMAGE_PATH}")
    print("\nTo get a sample:")
    print("1. Visit https://flipp.com")
    print("2. Open a Metro/Food Basics/No Frills flyer")
    print("3. Take a screenshot")
    print(f"4. Save to: {IMAGE_PATH}")

## 2. Extract Data with Claude Vision

In [ ]:
def extract_flyer_data(image_path: Path, api_key: str = None):
    """Extract items and prices using Claude Vision API."""
    
    # Load image
    with open(image_path, 'rb') as f:
        image_data = base64.b64encode(f.read()).decode()
    
    # Determine media type
    media_type = 'image/png' if image_path.suffix.lower() == '.png' else 'image/jpeg'
    
    # Initialize client
    client = anthropic.Anthropic(api_key=api_key) if api_key else anthropic.Anthropic()
    
    # Extraction prompt
    prompt = """Extract ALL items and prices from this retail flyer image.

For each item, extract:
- Product name (as shown on flyer)
- Price (numeric value)
- Unit (each, lb, kg, 100g, etc.)
- Original price (if on sale)
- Promotional details (e.g., "Buy 2 Get 1 Free", "Save $2", etc.)
- Confidence (0.0-1.0) based on text clarity

Return ONLY valid JSON in this exact format (no markdown, no code blocks):
{
  "metadata": {
    "total_items_found": 0,
    "page_quality": "high/medium/low",
    "extraction_notes": "any relevant observations"
  },
  "items": [
    {
      "name": "Product Name",
      "price": "5.99",
      "unit": "each",
      "original_price": "7.99",
      "promotion": "Save $2",
      "confidence": 0.95,
      "location": "top-left/center/etc"
    }
  ]
}

Rules:
1. Price must be numeric string (e.g., "5.99" not "$5.99")
2. Include ALL visible items, even if partially visible
3. Set confidence lower if text is blurry or unclear
4. Leave fields as null if not applicable
5. For multi-buy deals (e.g., "2 for $5"), calculate unit price if possible
6. Include brand names if visible
"""
    
    # Make API call
    print("⏳ Calling Claude API...")
    start_time = time.time()
    
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=4096,
        temperature=0,
        messages=[{
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": media_type,
                        "data": image_data
                    }
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }]
    )
    
    end_time = time.time()
    processing_time = end_time - start_time
    
    # Parse response
    response_text = message.content[0].text
    
    # Clean markdown if present
    if response_text.strip().startswith("```"):
        lines = response_text.strip().split('\n')
        response_text = '\n'.join(lines[1:-1]) if len(lines) > 2 else response_text
        if response_text.startswith('json'):
            response_text = response_text[4:]
    
    extracted_data = json.loads(response_text.strip())
    
    # Calculate costs
    input_tokens = message.usage.input_tokens
    output_tokens = message.usage.output_tokens
    
    # Claude 3.5 Sonnet pricing: $3/MTok input, $15/MTok output
    input_cost = (input_tokens / 1_000_000) * 3.00
    output_cost = (output_tokens / 1_000_000) * 15.00
    total_cost = input_cost + output_cost
    
    return {
        "data": extracted_data,
        "performance": {
            "time_seconds": round(processing_time, 2),
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "total_tokens": input_tokens + output_tokens,
            "cost_usd": round(total_cost, 4)
        },
        "raw_response": response_text
    }

In [ ]:
# Run extraction
if IMAGE_PATH.exists():
    result = extract_flyer_data(IMAGE_PATH, api_key=API_KEY)
    print("✅ Extraction complete!")
else:
    print("❌ No image found. Please add an image first.")

## 3. Analyze Results

In [ ]:
# Performance metrics
if 'result' in locals():
    perf = result['performance']
    
    print("="*70)
    print("📊 PERFORMANCE METRICS")
    print("="*70)
    print(f"\n⏱️  Processing Time: {perf['time_seconds']}s")
    print(f"🎫 Total Tokens: {perf['total_tokens']:,}")
    print(f"   - Input: {perf['input_tokens']:,} tokens")
    print(f"   - Output: {perf['output_tokens']:,} tokens")
    print(f"\n💰 Cost: ${perf['cost_usd']:.4f} per page")
    
    # Projections
    cost = perf['cost_usd']
    print(f"\n📈 Cost Projections:")
    print(f"   - 10-page flyer: ${cost * 10:.3f}")
    print(f"   - 100 flyers (12 pages avg): ${cost * 1200:.2f}")
    print(f"   - 1,000 flyers: ${cost * 12000:.2f}")

In [ ]:
# Extracted data
if 'result' in locals():
    data = result['data']
    
    print("="*70)
    print("📦 EXTRACTED DATA")
    print("="*70)
    
    if 'metadata' in data:
        meta = data['metadata']
        print(f"\n📋 Metadata:")
        print(f"   - Total items: {meta.get('total_items_found', 'N/A')}")
        print(f"   - Page quality: {meta.get('page_quality', 'N/A')}")
        if meta.get('extraction_notes'):
            print(f"   - Notes: {meta['extraction_notes']}")
    
    if 'items' in data:
        items = data['items']
        print(f"\n🛒 Items Found: {len(items)}")
        
        # Show first 10 items
        for i, item in enumerate(items[:10], 1):
            print(f"\n{i}. {item.get('name', 'Unknown')}")
            print(f"   💵 ${item.get('price', 'N/A')}")
            
            if item.get('unit'):
                print(f"   📦 {item['unit']}")
            
            if item.get('original_price'):
                print(f"   🏷️  Was: ${item['original_price']}")
            
            if item.get('promotion'):
                print(f"   🎉 {item['promotion']}")
            
            conf = item.get('confidence', 0)
            emoji = "🟢" if conf > 0.9 else "🟡" if conf > 0.7 else "🔴"
            print(f"   {emoji} Confidence: {conf:.0%}")
        
        if len(items) > 10:
            print(f"\n... and {len(items) - 10} more items")

In [ ]:
# Quality analysis
if 'result' in locals() and 'items' in result['data']:
    items = result['data']['items']
    
    print("="*70)
    print("🎯 QUALITY ASSESSMENT")
    print("="*70)
    
    confidences = [item.get('confidence', 0) for item in items]
    
    if confidences:
        avg_conf = sum(confidences) / len(confidences)
        
        high = sum(1 for c in confidences if c > 0.9)
        med = sum(1 for c in confidences if 0.7 < c <= 0.9)
        low = sum(1 for c in confidences if c <= 0.7)
        
        print(f"\n📊 Confidence Distribution:")
        print(f"   🟢 High (>90%): {high} items ({high/len(confidences)*100:.0f}%)")
        print(f"   🟡 Medium (70-90%): {med} items ({med/len(confidences)*100:.0f}%)")
        print(f"   🔴 Low (<70%): {low} items ({low/len(confidences)*100:.0f}%)")
        print(f"\n   📈 Average Confidence: {avg_conf:.1%}")
        
        # Price validation
        prices = [float(item.get('price', 0)) for item in items if item.get('price')]
        if prices:
            print(f"\n💰 Price Range:")
            print(f"   Min: ${min(prices):.2f}")
            print(f"   Max: ${max(prices):.2f}")
            print(f"   Avg: ${sum(prices)/len(prices):.2f}")

## 4. Export Results

In [ ]:
# Save to JSON
if 'result' in locals():
    output_path = Path("../data/processed/vision_test_results.json")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(result, f, indent=2)
    
    print(f"💾 Results saved to: {output_path}")
    
    # Also save just the items in a clean format
    items_path = Path("../data/processed/extracted_items.json")
    with open(items_path, 'w') as f:
        json.dump(result['data']['items'], f, indent=2)
    
    print(f"💾 Items saved to: {items_path}")

## 5. Compare with Manual Ground Truth (Optional)

Create a manual list of items you see in the flyer to compare accuracy.

In [ ]:
# Manual ground truth (fill in based on what you see in the image)
ground_truth = [
    {"name": "Bananas", "price": "0.69", "unit": "lb"},
    {"name": "Apples", "price": "1.99", "unit": "lb"},
    # Add more items...
]

if 'result' in locals() and ground_truth:
    extracted = result['data']['items']
    
    print(f"Manual count: {len(ground_truth)} items")
    print(f"Extracted count: {len(extracted)} items")
    print(f"Recall: {len(extracted) / len(ground_truth) * 100:.1f}%")

## Conclusion

**Key Findings:**
- Cost per page: Check the output above
- Items extracted: Check the output above
- Average confidence: Check the output above
- Processing time: Check the output above

**Next Steps:**
1. Test on 5-10 different flyers to get average accuracy
2. Compare cost vs. accuracy with Tier 1 (OCR + Regex) approach
3. Decide if vision API is worth the cost or if cheaper methods are sufficient
4. Consider hybrid approach: cheap methods first, vision API for low-confidence cases